In [6]:
import numpy as np
import matplotlib as plt
from sklearn import linear_model, datasets
import pymysql
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sshtunnel import SSHTunnelForwarder
import random
import functools
import pandas as pd

In [30]:
with SSHTunnelForwarder(
         ('1.212.234.245', 486),
         ssh_password="vips",
         ssh_username="vips",
         remote_bind_address=('127.0.0.1', 3306)) as server:

    conn = pymysql.connect(host='localhost',
                           port=server.local_bind_port,
                           user='root',
                           passwd='qlqtmajrwk',
                           db='dg',
                           charset='utf8')

    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "select * from dg.searchDB where (crp_cls = 'K' or crp_cls = 'Y')"
    curs.execute(sql)
    rcp_table = curs.fetchall()
    
    sql2 = "select * from dg.stockDB"
    curs.execute(sql2)
    stock_table = curs.fetchall()
    conn.commit()
    conn.close()
    
    
server.stop()

In [29]:
len(rcp_table) #2018-07-23 오전 update 기준

1295919

In [31]:
len(rcp_table)

1295926

# Rcp Table

In [238]:
rcp_pd = pd.DataFrame.from_dict(rcp_table)

In [239]:
rcp_pd.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12)
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12)
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12)
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12)
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12)


In [27]:
rcp_pd.groupby(['crp_cd','crp_nm']).size()

pandas.core.series.Series

In [73]:
def convertToKorean(x) :
    if x == 'Y': return '유가증권'
    elif x == 'K' : return '코스닥'
    else: return '코넥스'

In [74]:
rcp_pd['crp_cls'] = rcp_pd['crp_cls'].map(convertToKorean)

In [240]:
rcp_pd.groupby(['crp_cd','crp_cls','crp_nm']).size().to_frame().to_csv('기업별공시count.csv')

In [23]:
size

crp_cd  crp_nm    
000020  동화약품           792
000030  우리은행          1948
000040  KR모터스          770
000050  경방             684
000060  메리츠화재         1456
000070  삼양홀딩스          950
000080  하이트진로          874
000100  유한양행          1412
000120  CJ대한통운        1011
000140  하이트진로홀딩스      1091
000150  두산            2160
000180  성창기업지주         585
000210  대림산업          1909
000220  유유제약          1121
000230  일동홀딩스          872
000240  한국타이어월드와이드     967
000250  삼천당제약          382
000270  기아자동차         2638
000300  대유플러스          946
000320  노루홀딩스         1494
000370  한화손해보험        1409
000390  삼화페인트공업       1097
000400  롯데손해보험         912
000430  대원강업          1104
000440  중앙에너비스         438
000480  조선내화           585
000490  대동공업           526
000500  가온전선           567
000520  삼일제약           521
000540  흥국화재           962
                      ... 
289080  SV인베스트먼트        20
291210  한국제7호스팩          5
294870  HDC현대산업개발       24
298000  효성화학             9
298020  효성티앤씨           12
298040  효

In [99]:
rcp_pd['rcp_dt'][0].strftime("%y-%m")

'99-03'

In [89]:
rcp_pd.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12)
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12)
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12)
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12)
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12)


In [100]:
def toMonth(x):
    x.strftime("%y-%m")

In [126]:
#rcp_pd[rcp_dt]의 타입 변환 df.datetime으로
rcp_pd['rcp_dt'] = rcp_pd['rcp_dt'].apply(lambda df : datetime(year=df.year, month=df.month, day=df.day))

In [ ]:
date_index_pd = 

In [230]:
dt = pd.DatetimeIndex(rcp_pd.date)
rcp_pd['year'] = dt.year
rcp_pd['month'] = dt.month
rcp_pd['day'] = dt.day
rcp_pd['datecd'] = rcp_pd['year'].map(str)+rcp_pd['month'].map(str)+rcp_pd['day'].map(str)+rcp_pd['crp_cd']

In [193]:
rcp_pd.head()

,crp_cd,crp_cls,crp_nm,flr_nm,date,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,datecd,year,month,day
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12),19990 3\n1 3\n2 3\n...,1999,3,25
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12),19990 3\n1 3\n2 3\n...,1999,3,25
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12),19990 3\n1 3\n2 3\n...,1999,3,25
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12),19990 3\n1 3\n2 3\n...,1999,3,25
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12),19990 3\n1 3\n2 3\n...,1999,3,25


In [195]:
rcp_pd.groupby('month').size()

month
1      81082
2     127259
3     238189
4     107718
5     114876
6      83927
7      82189
8     101554
9      68596
10     76255
11    108573
12    105708
dtype: int64

In [196]:
rcp_pd.groupby('year').size()

year
1999      1102
2000     20387
2001     32075
2002     45489
2003     61364
2004     63287
2005     64288
2006     61674
2007     68520
2008     70355
2009     79657
2010     74374
2011     77785
2012     75321
2013     80175
2014     84694
2015     91806
2016     92725
2017    100678
2018     50170
dtype: int64

In [229]:
rcp_pd = rcp_pd.rename(columns = {'rcp_dt' : 'date'})

In [231]:
rcp_pd.head()

,crp_cd,now_diff,now_st,now_trd,now_volume,date,rcp_no,rmk,rpt_cat,rpt_link,rpt_nm,year,month,day,datecd
0,036420,140,7240,13969051000,1929063,2018-07-16 07:00:03,20180629100079,,None,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,효력발생안내( 2018.6.29. 제출 증권신고서(지분증권) ),2018,7,16,2018716036420
1,066110,-200,1900,2245463000,1178412,2018-07-18 07:00:02,20180703100010,,None,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,효력발생안내( 2018.7.3. 제출 증권신고서(지분증권) ),2018,7,18,2018718066110
2,008490,600,30750,407000000,13372,2018-07-16 07:00:03,20180704100007,,None,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,효력발생안내( 2018.7.4. 제출 증권신고서(채무증권) ),2018,7,16,2018716008490
3,065500,-3,625,258603000,416988,2018-07-19 07:00:05,20180704100008,,None,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,효력발생안내( 2018.7.4. 제출 증권신고서(지분증권) ),2018,7,19,2018719065500
4,043100,4,561,5695810000,9810253,2018-07-20 07:00:05,20180705100003,,None,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,효력발생안내( 2018.7.5. 제출 증권신고서(지분증권) ),2018,7,20,2018720043100


In [173]:
rcp_pd['date'][0]

Timestamp('1999-03-25 00:00:00')

# Close Table

In [77]:
close_pd = pd.DataFrame.from_dict(stock_table)

In [162]:
close_pd.head()

,close,crp_cd,date,datecd,diff,high,low,open,volume
0,36000.0,000020,1996-06-25,19960625000020,0.0,38400.0,36000.0,37700.0,102200.0
1,9300.0,000040,1996-06-25,19960625000040,0.0,9390.0,9290.0,9390.0,1600.0
2,42000.0,000050,1996-06-25,19960625000050,0.0,42000.0,42000.0,42000.0,0.0
3,30500.0,000060,1996-06-25,19960625000060,0.0,31000.0,29700.0,30100.0,68220.0
4,24700.0,000070,1996-06-25,19960625000070,0.0,25600.0,24700.0,25500.0,16960.0


In [236]:
dt = pd.DatetimeIndex(close_pd.date)
close_pd['year'] = dt.year
close_pd['month'] = dt.month
close_pd['day'] = dt.day
close_pd['datecd'] = close_pd['year'].map(str)+close_pd['month'].map(str)+close_pd['day'].map(str)+close_pd['crp_cd']

In [198]:
close_pd.head()

,close,crp_cd,date,datecd,diff,high,low,open,volume,year,month,day
0,36000.0,000020,1996-06-25,1996625000020,0.0,38400.0,36000.0,37700.0,102200.0,1996,6,25
1,9300.0,000040,1996-06-25,1996625000040,0.0,9390.0,9290.0,9390.0,1600.0,1996,6,25
2,42000.0,000050,1996-06-25,1996625000050,0.0,42000.0,42000.0,42000.0,0.0,1996,6,25
3,30500.0,000060,1996-06-25,1996625000060,0.0,31000.0,29700.0,30100.0,68220.0,1996,6,25
4,24700.0,000070,1996-06-25,1996625000070,0.0,25600.0,24700.0,25500.0,16960.0,1996,6,25


In [232]:
#datecd 기반으로 merge
merged_pd = close_pd.merge(rcp_pd, how = 'inner', on='datecd')

# to_csv

In [202]:
with SSHTunnelForwarder(
         ('1.212.234.245', 486),
         ssh_password="vips",
         ssh_username="vips",
         remote_bind_address=('127.0.0.1', 3306)) as server:

    conn = pymysql.connect(host='localhost',
                           port=server.local_bind_port,
                           user='root',
                           passwd='qlqtmajrwk',
                           db='yejinyDB',
                           charset='utf8')

    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "SELECT * FROM yejinyDB.Rcp where now_st != -1;"
    curs.execute(sql)
    zziny_rcp_table = curs.fetchall()
    
    conn.commit()
    conn.close()
    
    
server.stop()

ProgrammingError: (1146, "Table 'yejinyDB.Rcp' doesn't exist")

In [205]:
conn = pymysql.connect(host='localhost', user='root', password='qlqtmajrwk',
                       db='yejinyDB', charset='utf8')
 
# Connection 으로부터 Dictoionary Cursor 생성
curs = conn.cursor(pymysql.cursors.DictCursor)

In [209]:
sql = "SELECT * FROM yejinyDB.Rcp where now_st != -1;"
curs.execute(sql)
zziny_rcp_table = curs.fetchall()

conn.commit()
conn.close()
    
server.stop()

InterfaceError: (0, '')

In [218]:
zziny_rcp = pd.DataFrame.from_dict(zziny_rcp_table)

In [224]:
zziny_rcp.to_csv('실시간공시실시간주가.csv', encoding='utf-8')

In [235]:
merged_pd

,close,crp_cd_x,date_x,datecd,diff,high,low,open,volume,year_x,...,now_volume,date_y,rcp_no,rmk,rpt_cat,rpt_link,rpt_nm,year_y,month_y,day_y


In [234]:
merged_pd.to_csv('공시와회사merge.csv', encoding='utf-8')

In [241]:
close_pd = close_pd['year'=2017]

SyntaxError: invalid syntax (<ipython-input-241-b6b09a37e8ee>, line 1)